<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/lesson13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install torchmetrics
!pip install rouge
!pip install torch
import torch
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 120 kB 60.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.0 MB/s 
     |████████████████████████████████| 212 kB 57.3 MB/s 
     |████████████████████████████████| 115 kB 53.3 MB/s 
     |████████████████████████████████| 127 kB 58.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

In [2]:
from datasets import load_dataset
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 52400
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5770
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5265
    })
})

In [4]:
import pandas as pd
df_train = pd.DataFrame(dataset['train'])
df_train.head()

,text,summary,title,date,url
0,«По итогам 2011 года чистый отток может состав...,"В 2011 году из России уйдет $80 млрд, считают ...",Прогноз не успевает за оттоком,2011-11-30 18:33:39,https://www.gazeta.ru/financial/2011/11/30/385...
1,Российское подразделение интернет-корпорации G...,"Юлия Соловьева, экс-директор холдинга «Профмед...",Google закончил поиск,2013-01-24 18:20:09,https://www.gazeta.ru/business/2013/01/24/4939...
2,Басманный районный суд Москвы вечером 6 феврал...,Суд арестовал на два месяца четверых экс-чинов...,«Фигуранты дела могут давить на свидетелей»,2018-02-06 21:21:14,https://www.gazeta.ru/social/2018/02/06/116393...
3,Как повлияло вступление в ВТО на конкурентносп...,Мнения предпринимателей по поводу вступления в...,«С последних традиционно «отжимают» больше»,2013-06-21 17:43:50,https://www.gazeta.ru/business/2013/06/21/5388...
4,К третьему сезону «Голос» на Первом канале ста...,На Первом канале завершился третий сезон шоу «...,Третий «Голос» за Градского,2014-12-27 01:10:01,https://www.gazeta.ru/culture/2014/12/27/a_636...


In [5]:
df_val = pd.DataFrame(dataset['validation'])
df_val.head()

,text,summary,title,date,url
0,"После громких приобретений Андре Шюррле, Гуса ...",Московский «Спартак» продолжает активную транс...,Замена Фернандо: кого может купить «Спартак»,2019-08-11 23:14:58,https://www.gazeta.ru/sport/2019/08/11/a_12572...
1,Американское издание The National Interest оце...,Издание The National Interest оценило перспект...,Шестое поколение: в США оценили российский бом...,2019-08-11 21:16:04,https://www.gazeta.ru/army/2019/08/11/12572317...
2,Министр иностранных дел России Сергей Лавров с...,"Глава МИД России Сергей Лавров заявил, что в 2...",«Зачем гонять людей?»: США требуют новый рефер...,2019-06-06 09:47:35,https://www.gazeta.ru/politics/2019/06/06_a_12...
3,Минфин предложил с января 2020 года увеличить ...,Министерство финансов предлагает вдвое поднять...,Перейти на отечественное: что будет с зарубежн...,2019-07-24 20:38:15,https://www.gazeta.ru/business/2019/07/24/1252...
4,Заявление командующего военно-воздушными силам...,Американские ПВО провалились в Саудовской Арав...,«Глупое заявление»: Медведев напомнил о провал...,2019-09-20 21:16:22,https://www.gazeta.ru/army/2019/09/20/12664933...


In [6]:
df_test = pd.DataFrame(dataset['test'])
df_test.head()

,text,summary,title,date,url
0,Американское аэрокосмическое агентство NASA ог...,"В NASA назвали четыре миссии в дальний космос,...","Венера, Ио или Тритон: куда полетит NASA",2020-02-14 16:39:11,https://www.gazeta.ru/science/2020/02/14_a_129...
1,Около 11 тысяч зрителей увидели все самое лучш...,25 и 26 февраля в Кремлевском дворце съездов п...,«Люди в Бурятии очень талантливые»,2020-02-28 10:44:13,https://www.gazeta.ru/social/2020/02/28/129806...
2,7 ноября в Белоруссии прошли выборы членов сов...,В Белоруссии в день годовщины Октябрьской рево...,Вспомнить СССР: как Лукашенко провел выборы,2019-11-07 19:55:08,https://www.gazeta.ru/politics/2019/11/07_a_12...
3,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,2020-03-01 16:50:06,https://www.gazeta.ru/culture/2020/03/01/a_129...
4,Депутат Верховной рады от партии «Слуга народа...,Украина не должна выплачивать пенсии жителям Д...,«Поддерживают Россию»: почему Киев не платит п...,2020-02-06 12:41:24,https://www.gazeta.ru/business/2020/02/06/1294...


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "IlyaGusev/rugpt3medium_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

article_text = "..."

text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False, 
    padding=False,
    truncation=True
)["input_ids"]
input_ids = text_tokens + [tokenizer.sep_token_id]
input_ids = torch.LongTensor([input_ids])

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)

summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
summary = summary.split(tokenizer.sep_token)[1]
summary = summary.split(tokenizer.eos_token)[0]
print(summary)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 800 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Нападающий «Зенита» Александр Кокорин и полузащитник «Краснодара» Павел Мамаев, обвиняемые в избиении водителя ведущей Первого канала Ольги Ушаковой, отпущены под подписку о невыезде. Об этом сообщил адвокат Кокорина Игорь Бушманов.


In [8]:
#функция подсчета метрик
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [9]:
#подсчет метрик на примере предсказания модели
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
target = sent_tokenize(df_test['summary'][1])
preds = sent_tokenize(summary)
calc_scores(target, preds)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Count: 2
Ref: Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Hyp: Об этом сообщил адвокат Кокорина Игорь Бушманов.
BLEU:  0.05879762398503692
ROUGE:  {'rouge-1': {'r': 0.07142857142857142, 'p': 0.043478260869565216, 'f': 0.05405405170197235}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.07142857142857142, 'p': 0.043478260869565216, 'f': 0.05405405170197235}}
